In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
from evallm.experiments.evaluate_tokenization_regexp_non_icl import (
    table_of_results,
    text_table,
    latex_table,
)

In [3]:
table = table_of_results()

In [4]:
non_response_threshold = 0.98
non_responses = table[table.non_response > non_response_threshold]
print(
    "Models",
    ", ".join(non_responses.index),
    f"have non-response rates over {non_response_threshold:.0%}".replace("%", "\\%"),
)

Models mistral-nemo-minitron-8B, mistral-nemo-base-12B, falcon-7b, deepseek-coder-33b-instruct have non-response rates over 98\%


In [5]:
text_table(table, non_response_threshold)

llama3-8B                       66%
llama3-70B                      80% [Non-response: 26%]
llama3.1-8B-Instruct            83%
llama3.1-8B                     89%
llama3.1-70B                    86% [Non-response: 36%]
qwen-2.5-7B                     62%
qwen-2.5-32B                   100%
mistral-nemo-instruct-12B       99%
gemma-7b                        67% [Non-response: 15%]
starcoder2-15b                  62%
codestral-22B                   95%
qwen-2.5-coder-7B              100%
qwen-2.5-coder-instruct-7B      56%
qwen-2.5-coder-instruct-32B     96%
gpt-3.5-instruct                67%
gpt-3.5-chat                    82%
gpt-4o-mini                    100%
gpt-4o                         100%
claude-3.5                     100%
o3-mini                        100%


In [6]:
latex_table(table, non_response_threshold)

\begin{tabular}{l r r}
\toprule
Model & Accuracy & Non-response \\
\midrule
llama3-8B & 66\% &  \\
llama3-70B & 80\% & 26\% \\
llama3.1-8B-Instruct & 83\% &  \\
llama3.1-8B & 89\% &  \\
llama3.1-70B & 86\% & 36\% \\
qwen-2.5-7B & 62\% &  \\
qwen-2.5-32B & 100\% &  \\
mistral-nemo-instruct-12B & 99\% &  \\
gemma-7b & 67\% & 15\% \\
starcoder2-15b & 62\% &  \\
codestral-22B & 95\% &  \\
qwen-2.5-coder-7B & 100\% &  \\
qwen-2.5-coder-instruct-7B & 56\% &  \\
qwen-2.5-coder-instruct-32B & 96\% &  \\
gpt-3.5-instruct & 67\% &  \\
gpt-3.5-chat & 82\% &  \\
gpt-4o-mini & 100\% &  \\
gpt-4o & 100\% &  \\
claude-3.5 & 100\% &  \\
o3-mini & 100\% &  \\
\bottomrule
\end{tabular}
